# Goals for today
- Defining new types in Julia
- Abstract types
- Generic code

# Defining new types in Julia

In [1]:
mutable struct MyDiscreteWalker # -- camel case
    x::Int64
end

Julia automatically created 2 **constructors** &rarr; functions to create objects of that types

In [2]:
methods(MyDiscreteWalker)

# 2 methods for type constructor:
[1] MyDiscreteWalker(x::Int64) in Main at In[1]:2
[2] MyDiscreteWalker(x) in Main at In[1]:2

In [3]:
w = MyDiscreteWalker(3)

MyDiscreteWalker(3)

In [4]:
typeof(w)

MyDiscreteWalker

In [5]:
Complex(3, 5)

3 + 5im

In [6]:
@which Complex(3,5)

Complex(re::T, im::T) where T<:Real in Base at complex.jl:12

In [7]:
w isa MyDiscreteWalker # w is an instance of the type MyDiscreteWalker

true

`MyDiscreteWalker(x::Int64)` is a method that works *only* when it receives an argument `x` of type `Int64`

`MyDiscreteWalker(x)` will accept an `x` of any type

In [8]:
MyDiscreteWalker(4.0)

MyDiscreteWalker(4)

In [9]:
MyDiscreteWalker(4.1)

LoadError: InexactError: Int64(4.1)

Collecting information into one packet that belongs together: **encapsulation**

In [10]:
mutable struct Agent
    infection_state::Int
    num_infected::Int
end

More common to use *immutable* structs:

In [11]:
struct ExampleImmutable
    x::Int
    y::Int
end

In [12]:
z = ExampleImmutable()

LoadError: MethodError: no method matching ExampleImmutable()
[0mClosest candidates are:
[0m  ExampleImmutable([91m::Int64[39m, [91m::Int64[39m) at In[11]:2
[0m  ExampleImmutable([91m::Any[39m, [91m::Any[39m) at In[11]:2

In [13]:
convert(Int64, 3.0)

3

In [14]:
convert(Int64, 3.1)

LoadError: InexactError: Int64(3.1)

In [15]:
@which Int64(3.0)

Int64(x::Float64) in Base at float.jl:719

In [16]:
z = ExampleImmutable(1,2)

ExampleImmutable(1, 2)

In [17]:
z.x

1

In [18]:
z.y

2

In [19]:
MyDiscreteWalker() = MyDiscreteWalker(0)

MyDiscreteWalker

In [20]:
methods(MyDiscreteWalker) # outer constructor -- lives outside definition of type

# 3 methods for type constructor:
[1] MyDiscreteWalker() in Main at In[19]:1
[2] MyDiscreteWalker(x::Int64) in Main at In[1]:2
[3] MyDiscreteWalker(x) in Main at In[1]:2

In [21]:
w = MyDiscreteWalker()

MyDiscreteWalker(0)

In [22]:
w.x

0

## Make walker move

In [23]:
function jump!(w::MyDiscreteWalker) 
    w.x += rand( (-1,+1) )
end

jump! (generic function with 1 method)

In [24]:
pos(w::MyDiscreteWalker) = w.x # getter function

pos (generic function with 1 method)

In [25]:
propertynames(w)

(:x,)

In [26]:
pos(w) # interface to my object; removes me from the internal details

0

In [27]:
function set_pos!(w,x) # setter function
    w.x = x
end

set_pos! (generic function with 1 method)

In [28]:
jump(MyDiscreteWalker) = rand( (-1,+1) )

jump (generic function with 1 method)

In [36]:
function jump!(w::MyDiscreteWalker)
    old_pos = pos(w)
    set_pos!(w, old_pos + jump(w))
end

jump! (generic function with 1 method)

In [37]:
function walk!(w::MyDiscreteWalker, N)
    for i in 1:N
        jump!(w)
    end
    
    return w
end

walk! (generic function with 1 method)

In [38]:
w = MyDiscreteWalker()

MyDiscreteWalker(0)

In [39]:
walk!(10)

LoadError: MethodError: no method matching walk!(::Int64)
[0mClosest candidates are:
[0m  walk!([91m::MyDiscreteWalker[39m, [91m::Any[39m) at In[37]:1

In [40]:
walk!(w, 10)

MyDiscreteWalker(-2)

In [41]:
pos(w)

-2

In [42]:
methods(set_pos!)

# 1 method for generic function "set_pos!":
[1] set_pos!(w, x) in Main at In[27]:1

## Continuous walkers

In [43]:
mutable struct MyContinuousWalker
    y::Float64
end

In [44]:
w = MyContinuousWalker(3)

MyContinuousWalker(3.0)

In [46]:
jump(w::MyContinuousWalker) = rand()

jump (generic function with 2 methods)

In [55]:
function jump!(w) # takes argument w of *any* type
    old_pos = pos(w)
    set_pos!(w, old_pos + jump(w))
end

jump! (generic function with 2 methods)

In [56]:
w

MyContinuousWalker(3.0)

In [57]:
jump!(w)

3.172907019291668

In [58]:
pos(w::MyContinuousWalker) = w.y

pos (generic function with 2 methods)

In [59]:
function set_pos!(w::MyContinuousWalker, pos)
    w.y = pos
end

set_pos! (generic function with 2 methods)

In [60]:
jump!(w)

3.942493598846559

In [61]:
walk!(w, 10)

LoadError: MethodError: no method matching walk!(::MyContinuousWalker, ::Int64)
[0mClosest candidates are:
[0m  walk!([91m::MyDiscreteWalker[39m, ::Any) at In[37]:1

In [62]:
function walk!(w, N)
    for i in 1:N
        jump!(w)
    end
    
    return w
end

walk! (generic function with 2 methods)

In [63]:
walk!(w,20)

MyContinuousWalker(14.575240221760486)

In [64]:
z = MyDiscreteWalker()

MyDiscreteWalker(0)

In [65]:
walk!(z,10)

MyDiscreteWalker(2)

## Abstract types

" A discrete walker is a **kind of** walker"

In [66]:
abstract type RandomWalker end

In [67]:
RandomWalker

RandomWalker

In [68]:
methods(RandomWalker)

# 0 methods for type constructor:

In [70]:
mutable struct DiscreteWalker <: RandomWalker
    x::Int64
end

In [73]:
mutable struct ContinuousWalker <: RandomWalker
    y::Int64
end

In [75]:
pos(w::DiscreteWalker) = w.x
pos(w::ContinuousWalker) = w.y

pos (generic function with 4 methods)

In [76]:
function walk!(w::RandomWalker, N)
    for i in 1:N
        jump!(w)
    end
    
    return w
end

walk! (generic function with 3 methods)

In [78]:
w = DiscreteWalker(10)

DiscreteWalker(10)

In [79]:
w isa DiscreteWalker

true

In [81]:
w isa RandomWalker

true